In [1]:
# Importing necessary libraries
import sys, os

# Setting the root directory as a string.
root = r'<ROOT LOCATION OF REPORT FOLDER>'

# Combining the root directory with the target directory to create the full path
path = os.path.join(root, "targetdirectory")

# Create empty lists to store the full file paths and file names
fullFilePaths = []
fileNames = []

# Walk through all the subdirectories and files within the root directory
for path, subdirs, files in os.walk(root):
    # Loop through all the files in each subdirectory
    for name in files:
        # Create the full file path by joining the path and file name
        fullFilePaths.append(os.path.join(path, name))

# The final output will be the full file paths of all the files within the specified root directory.

In [2]:
# Import the pandas library to work with dataframes
import pandas as pd

# Specify the filepath of the GRI excel file using a raw string to prevent backslash escaping
griFullFileName = r'<GRI FILE LOCATION>'

# Open the GRI excel file using pd.ExcelFile() and save it to the variable xls
xls = pd.ExcelFile(griFullFileName)

# Create an empty dictionary to store the dataframes
fullGRI = {}

# Loop through each sheet in the Excel file except for the "Overview" sheet
# and read the sheet into a dataframe using pd.read_excel()
# Add each dataframe to the fullGRI dictionary with the sheet name as the key
for sheet_name in xls.sheet_names[1:21]:
    fullGRI[sheet_name] = pd.read_excel(griFullFileName, sheet_name = sheet_name, header=1)

# Read the "Overview" sheet into a dataframe separately and add it to the fullGRI dictionary with the key "Overview"
fullGRI["Overview"] = pd.read_excel(griFullFileName, sheet_name = "Overview")

# Create a new dataframe named total_df by concatenating all dataframes in the fullGRI dictionary except for the "Overview" sheet
# Set ignore_index argument to True to create a new index for the concatenated dataframe
total_df = fullGRI["1999"]
for sheet_name in xls.sheet_names[2:21]:
    total_df = pd.concat([total_df, fullGRI[sheet_name]], ignore_index=True)

# Modify the 'Name' column in total_df by removing all spaces using the str.replace() method
total_df['Name'] = total_df['Name'].str.replace(' ', '')

# The resulting concatenated dataframe total_df contains all GRI data from 1999 to 2018

In [3]:
# Import the regular expressions library to work with patterns
import re

# Create an empty dataframe with two columns named "Organization" and "Year"
df = pd.DataFrame(columns=["Organization", "Year"])

# Loop through each file path in the fullFilePaths list
for filePath in fullFilePaths:
    # Extract the file name without extension from the file path
    fileName = os.path.splitext(os.path.split(filePath)[1])[0]

    # Create a list of two items by splitting the file name using "_" as a separator
    # The first item is the organization name, and the second item is the year
    row = [fileName.split("_")[0], fileName.split("_")[-1]]

    # Add the row to the dataframe at the next available index using df.loc[]
    df.loc[len(df)] = row

# Convert the "Year" column to numeric values using pd.to_numeric()
df['Year'] = pd.to_numeric(df['Year'])

# The resulting dataframe df contains information about the PDF files, including the organization name and year

In [4]:
# INPUT: GRI DATAFRAME and PDF FILES DATAFRAME
import numpy as np

# merge the two dataframes based on 'Organization' and 'Year' columns
almost_df = pd.merge(df, total_df,  how='left', left_on=['Organization','Year'], right_on = ['Name','Publication Year'])

# drop the 'Name' and 'Publication Year' columns from the merged dataframe
final_df = almost_df.drop(['Name','Publication Year'], axis=1)

# drop duplicate rows based on 'Organization' and 'Year' columns
final_df = final_df.drop_duplicates(subset=['Organization', 'Year']).reset_index(drop=True)

# OUTPUT: ONE FULL MERGED DATAFRAME WITH ALL THE PDF FILES AND GRI INFORMATION

In [5]:
# This code reads several CSV files and creates pandas dataframes from them. 
# Each dataframe contains specific columns of interest from their respective CSV files. 
# The file paths of the CSV files are hardcoded into the code.
df_keywords = pd.read_csv(r'<FILE LOCATION OF KEYWORD RESEARCH>')[["Greenhouse_Gas_Emissions", "Diversity", "Employee_Health_Safety", "Customer_Welfare"]]
df_language = pd.read_csv(r'<FILE LOCATION OF LANGUAGE RESEARCH>')[["Language"]]
df_sentiment = pd.read_csv(r'<FILE LOCATION OF SENTIMENT RESEARCH>')[["neg", "neu", "pos", "compound", "Overall sentiment"]]
df_topicModelling = pd.read_csv(r'<FILE LOCATION OF TOPIC MODELLING RESEARCH>')[["Topics"]]
df_wordCount = pd.read_csv(r'<FILE LOCATION OF WORDCOUNT RESEARCH>')[["WordCount"]]
# The resulting dataframes will be used later in the program, to combine the data from all of the dataframes into a single dataframe that contains all of the desired information.

In [6]:
# Here all the different dataframes are combined onto eachother
final_df = final_df.join(df_keywords).join(df_language).join(df_sentiment).join(df_topicModelling).join(df_wordCount)

In [7]:
# Define the file location to which the final dataframe will be exported
file_location = r'<EXPORT FILE LOCATION>'

# Export the final dataframe to the defined file location as a CSV file, without the index column
final_df.to_csv(file_location, index=False)